In [13]:
! pip install numpy
import numpy as np

In [14]:
# Market parameters
S0 = 1; # spot exchange rate
# Contract parameters
T = 1; # maturity
K = 1; # strike price
y_0 = np.array([0.05,0.06,0.1244,0.0591]);
# Interest rate coefficients or weights
a_i = np.array([1.004,000000,0.6650, 1.0985]);
a_j = np.array([0000000,1.0006,1.6177, 1.3588]);
# Mean-reversion rate (or strength) of the interest rate
kappa = np.array([0.02,0.02,0.9418,1.7909]);
# Long-term average of the interest rate
y_bar = np.array([0.05,0.06,0.037,0.0909]);
# Volatility of the interest rate
sigma = np.array([0.002,0.002,0.4912,0.08]);
# Correlations
rho = np.array([0.5231,-0.398,-0.23,-0.81]);
# Auxiliary parameters
a_ij_minus = a_i - a_j;
a_ij_plus = a_i + a_j;
a_ij_rho = np.multiply(rho,a_ij_minus);
a_ij_division = np.divide(a_ij_plus,a_ij_minus);
h = np.array([1,-1,0,0]);

d = 4;
# Number of simulations
nsimulations = 500;
# Number of steps
nsteps = 25;
dt = T/nsteps;
# Initialize calls
VcMC = np.zeros(nsimulations);
VpMC = np.zeros(nsimulations);

for simulation in range(nsimulations):
    y = np.zeros((nsteps+1,d));
    y[0,:] = y_0;
    # corr (dW_v, dZ_v) = rho_v
    dW_1 = np.random.normal(size=(nsteps,d));
    dW_2 = np.random.normal(size=(nsteps,d));
    dW_3 = dW_1*rho + dW_2*(1-rho**2)**0.5;
    dW = dW_1 * np.sqrt(dt);
    dZ = dW_3 * np.sqrt(dt);

    for step in range(nsteps):
        y[step+1,:] = np.maximum((y[step,:] + (y_bar - y[step,:]) * kappa * dt + np.sqrt(y[step,:]) * dZ[step,:]* sigma),0);

    sum_y_1 = np.zeros((nsteps,1));
    sum_y_2 = np.zeros((nsteps,1));
    mu = np.zeros((nsteps,1));
    x = np.zeros((nsteps+1,1));

    for step in range(nsteps):
        sum_y_1[step] = y[step,:] * (a_i**2 - a_j**2);
        mu[step] = np.inner(y[step,:],h) + 0.5 * (sum_y_1[step]);
        sum_y_2[step] = y[step,:]**0.5 * dW[step,:] * [a_i-a_j];
        x[step+1] = x[step] + mu[step]*dt + sum_y_2[step];

    S_end = S0*np.exp(x(-1));

    payoffs_call = max(S_end - K,0);
    payoffs_put = max(K - S_end,0);

    VcMC[simulation] = np.exp(-y_0(1)*T)*payoffs_call;
    VpMC[simulation] = np.exp(-y_0(1)*T)*payoffs_put;

simulated_call = np.mean(VcMC);
simulated_put = np.mean(VpMC);


ValueError: could not broadcast input array from shape (4,) into shape (1,)